In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.decomposition import PCA

<span style="color: red;">
<div dir='rtl'>

<b> نام و نام‌‌خانوادگی: سید احسان حسن بیگی</b>

<b> شماره دانشجویی: ۴۰۲۲۱۱۷۲۳</b>

</span>

<div dir='rtl'>
<h1> 
مقدمه
</h1>
<p>
در این تمرین قصد داریم به مباحث زیر بپردازیم:
<li> کاهش ابعاد </li>
<li> خوشه‌بندی متن و هرس‌کردن خوشه‌ها</li>

کتابخانه‌های مورد نظرتان را هم می‌توانید در اولین سل نوت‌بوک فراخوانی کنید. 

تمرین درباره ی خوشه بندی اهنگ های مشابه در اپ اسپاتیفای است که از دیتاستی از سایت کگل برداشته شده است.
</p>
</div>

<div dir="rtl">
<h1>
لود دیتا</h1>
</div>

<div dir="rtl">
در این بخش باید دیتا را لود کرده و در انتها فقط نام آهنگ و فیچر هایی که به نظرتان مفید است را با ذکر علتی کوتاه نگه دارید. 
</div>

In [3]:
data = pd.read_csv("datasets/spotify.csv")
data = data.dropna()

# merge rows with the same 'track_id' and replace numeric columns with the mean
# replace numeric columns with the mean
# replace non-numeric columns with the most frequent value
# numeric_columns = data.select_dtypes(include=["number"]).columns.tolist()
# non_numeric_columns = data.select_dtypes(exclude=["number"]).columns.tolist()
# data = data.groupby("track_id", as_index=False).agg(
#     {
#         **{col: "mean" for col in numeric_columns},
#         **{col: lambda x: x.mode().iloc[0] for col in non_numeric_columns},
#     }
# )

garbage_cols = ["track_id", "track_album_id", "playlist_id", "playlist_name"]
data = data.drop(columns=garbage_cols, axis=1)

Index(['track_name', 'track_artist', 'track_popularity', 'track_album_name',
       'track_album_release_date', 'playlist_genre', 'playlist_subgenre',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms'],
      dtype='object')


In [4]:
non_numeric_columns = data.select_dtypes(exclude=["number"]).columns.tolist()
for col in non_numeric_columns:
    le = preprocessing.LabelEncoder()
    data[col] = le.fit_transform(data[col])
    if col == 'track_name':
        track_name_le = le

# data['track_name'] = track_name_le.inverse_transform(data['track_name'])

<div dir="rtl">
توضیح دهید که دیتای scale نشده چه مشکلی میتواند برای خوشه بندی ایجاد کند?
<br>
در این cell باید یک standard scalar را از صفر پیاده سازی نمایید. تا برای قسمت های بعدی یک دیتای مناسب داشته باشید.
</div>

In [6]:
# standardization
def standard_scalar(df, numeric_columns=None):
    if numeric_columns is None:
        numeric_columns = df.select_dtypes(include=["number"]).columns.tolist()

    column_stats = {}
    for col in numeric_columns:
        mean = df[numeric_columns][col].mean()
        std = df[numeric_columns][col].std()
        column_stats[col] = {'mean': mean, 'std': std}

        df[col] = (df[col] - mean) / std

    return df, column_stats

<div dir="rtl">
<h1>
کاهش ابعاد
</h1>
</div>

<div dir="rtl">
یکی از روش‌های کاهش ابعاد، PCA است. با استفاده از پیاده‌سازی آن در کتابخانه sklearn، ابعاد ویژگی‌ را کاهش دهید.
<br>
سپس با استفاده از explained_variance_ratio_ در الگوریتم PCA  نشان دهید که با وجود یک ترشولد مناسب  تا چه میزان میتوان ابعاد ویژگی ها را کم تر کرد.
<br>
برای ترشولدی که انتخاب کردید دلیل بیاورید و شهود این متغیر را توضیح دهید.
</div>

In [7]:
def reduce_dimension(embedding, n_components):
    """
    Performs dimensional reduction using PCA with n components left behind

    Parameters
    ----------
    embeddings : List
        A list of embeddings of documents
    
    n_components: int
        Number of components to keep

    Returns a list of reduced embeddings
    """
    pca = PCA(n_components)
    pca.fit(embedding)
    return pca.transform(embedding)

In [8]:
column_data_types = data.dtypes
data_type_counts = column_data_types.value_counts()
print(data_type_counts)

# print(data.columns)
# print(reduce_dimension(data, .95))
# print(data.columns)

AttributeError: 'dict' object has no attribute 'dtypes'

In [ ]:
# TODO: Findout the most reduced dimension which has a high cutoff explained variance. Write down the value of cut off and explain it.

<div dir="rtl">
<h2>
خوشه‌بندی
</h2>
</div>

<div dir="rtl">
در این قسمت، شما ابتدا الگوریتم خوشه‌بندی K-means را 
<u><b>از پایه</b></u>
 پیاده‌سازی می‌کنید.

</div>

In [ ]:

def cluster_kmeans(emb_vecs, n_clusters):
    """
    Clusters input vectors using K-means method

    Parameters
    ----------
    emb_vecs : List
        A list of vectors
    
    n_clusters: int
        Number of clusters

    Returns
    -------
    list
        Two lists: 1) A list containing cluster centers 2) A list containing cluster index for each input vector
    """

    # TODO: Implement K-means method from scratch
    # You can't use sklearn.cluster.KMeans for clustering
    # implement kmeans clustering here
    emb_matrix = None

    # Randomly initialize centroids
    initial_centroids = None

    for _ in range(None):  # You can adjust the number of iterations
        # Compute distances between data points and centroids
        distances = None

        # Assign each point to the closest centroid
        cluster_indices = None

        # Update centroids
        new_centroids = []
        

    return initial_centroids.tolist(), cluster_indices.tolist()

<div dir="rtl">
 با استفاده از K-Means خوشه‌ها را ایجاد کنید. الگوریتم را با استفاده از چند مقدار مختلف تعداد خوشه‌ها (k) اجرا کنید. در هربار اجرا، با استفاده از تعدادی از اسناد موجود در هر خوشه، موضوع آن خوشه را تعیین کرده و خوشه‌بندی حاصله را با استفاده از بردار‌های دوبعدی قسمت قبل، رسم کنید. با اینکار، پیاده‌سازی خود و همچنین کارایی این الگوریتم در خوشه‌بندی اسناد و قرار دادن اسناد مشابه در خوشه‌های یکسان را بررسی کنید.
<br>

 نمودار silhouette score برای مقدار‌های مختلف k را رسم کرده و silhouette analysis برای انتخاب k مناسب انجام دهید. 
<br>
همچنین Within-Cluster Sum of Squares (WSS) را نیز محاسبه کرده و در نهایت این دو کار بهترین مقدار k  را بدست بیاورید.
</div>

In [ ]:

def wss_score():
    #TODO
    pass

def silhouette_score():
    #TODO
    pass

In [ ]:
# TODO: Calculate silhouette score and wss score

In [ ]:
# TODO: Plot silhouette score for different value of k 

In [ ]:
#TODO: plot wss for different value of k 

<div dir="rtl">
با استفاده از نمودارهای رسم شده توضیح دهید بهترین k برای انتخاب در داده ما با استفاده از الگوریتم K-Means چیست؟
چرا؟
</div>

<span style="color: red;">#TODO: Write your answer in here.</span>

<div dir="rtl">
<h2>
بررسی خروجی</h2>
</div>

<div dir="rtl">
در این قسمت برای اینکه ببینیم چقدر خوب خوشه بندی را انجام داده ایم, از یک روش sample check و نیز TSNE استفاده خواهیم کرد.

</div>

<div dir="rtl">
در ابتدا به صورت رندوم از هر خوشه دو اهنگ را انتخاب کرده (با توجه به اینکه در بخش اول بایستی اسم اهنگ را نیز نگه میداشتید) و ببینید واقعا این اهنگ ها به هم نزدیک هستند یا خیر.
نکته اینکه صرفا بررسی این عمل نمره دارد و نیازی به خیلی دقیق بودن خروجی نیست.
</div>

In [ ]:
# TODO: Sample 2 songs from each cluster and compare them.

<div dir="rtl">
با استفاده از این T-SNE و کاهش ابعاد به 2 بعد نشان دهید که ایا خوشه بنده به خوبی انجام شده است یا خیر.

</br>
در صورت پیاده سازی تابع tsne  از ابتدا, نمره ی امتیازی به شما تعلق خواهد گرفت.
</div>

In [ ]:
# TODO: Visualize the clusters 

In [ ]:
# Bonus TODO: Implement TSNE from scratch